In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# Function for simulating Geometric Brownian Motion
def gbm(n_years=10, n_scenarios=1000, mu=0.07, sigma=0.15, steps_per_year=12, s_0=100.0, prices=True):
    dt = 1 / steps_per_year
    n_steps = int(n_years * steps_per_year) + 1
    rets_plus_1 = np.random.normal(loc=(1 + mu)**dt, scale=(sigma * np.sqrt(dt)), size=(n_steps, n_scenarios))
    rets_plus_1[0] = 1
    prices = s_0 * pd.DataFrame(rets_plus_1).cumprod() if prices else rets_plus_1 - 1
    return prices

# Function for applying the CPPI strategy
def cppi(gbm_sim, floor=80, multiplier=3, risk_free_rate=0.02, rebalance_per_year=12):
    n_steps, n_scenarios = gbm_sim.shape
    
    # Calculer le nombre d'étapes entre les rééquilibrages
    steps_per_rebalance = max(1, int(gbm_sim.index.size / (rebalance_per_year * (gbm_sim.index[-1] / 12))))
    
    account_value = pd.DataFrame(index=gbm_sim.index, columns=gbm_sim.columns)
    cushion = pd.DataFrame(index=gbm_sim.index, columns=gbm_sim.columns)
    risky_weight = pd.DataFrame(index=gbm_sim.index, columns=gbm_sim.columns)
    
    account_value.iloc[0] = gbm_sim.iloc[0]
    
    for step in range(1, n_steps):
        # Vérifier si c'est un moment de rééquilibrage
        if step % steps_per_rebalance == 0:
            cushion.iloc[step - 1] = np.maximum(account_value.iloc[step - 1] - floor, 0)
            risky_weight.iloc[step - 1] = multiplier * cushion.iloc[step - 1] / account_value.iloc[step - 1]
            risky_weight.iloc[step - 1] = np.clip(risky_weight.iloc[step - 1], 0, 1)
        else:
            risky_weight.iloc[step - 1] = risky_weight.iloc[step - 2] if step > 1 else 0
        
        safe_weight = 1 - risky_weight.iloc[step - 1]
        account_value.iloc[step] = (
            account_value.iloc[step - 1] * 
            (risky_weight.iloc[step - 1] * (gbm_sim.iloc[step] / gbm_sim.iloc[step - 1]) +
             safe_weight * (1 + risk_free_rate / rebalance_per_year))
        )
    
    return account_value

In [ ]:
# Interactive widgets
n_years_widget = widgets.IntSlider(value=5, min=1, max=20, step=1, description='Years')
n_scenarios_widget = widgets.IntSlider(value=100, min=10, max=1000, step=10, description='Scenarios')
mu_widget = widgets.FloatSlider(value=0.07, min=0.0, max=0.2, step=0.01, description='Mu')
sigma_widget = widgets.FloatSlider(value=0.15, min=0.0, max=0.5, step=0.01, description='Sigma')
steps_per_year_widget = widgets.IntSlider(value=255, min=12, max=365, step=1, description='Steps/Year')
s_0_widget = widgets.FloatSlider(value=100, min=50, max=500, step=10, description='Initial Price')

floor_widget = widgets.FloatSlider(value=90, min=50, max=100, step=1, description='Floor')
multiplier_widget = widgets.IntSlider(value=4, min=1, max=10, step=1, description='Multiplier')
rf_rate_widget = widgets.FloatSlider(value=0.04, min=0.0, max=0.1, step=0.01, description='Risk-free Rate')
rebalance_widget = widgets.IntSlider(value=10, min=1, max=255, step=1, description='Rebalance/Year')

# Button to run the simulation
run_button = widgets.Button(description="Run Simulation", button_style='success')

# Output widget to display results
output = widgets.Output()

# Function to run the simulation and plot results
def run_simulation(event):
    with output:
        output.clear_output()
        
        # Récupérer les valeurs des widgets
        n_years = n_years_widget.value
        n_scenarios = n_scenarios_widget.value
        mu = mu_widget.value
        sigma = sigma_widget.value
        steps_per_year = steps_per_year_widget.value
        s_0 = s_0_widget.value
        
        floor = floor_widget.value
        multiplier = multiplier_widget.value
        rf_rate = rf_rate_widget.value
        rebalance_per_year = rebalance_widget.value
        
        # Exécuter la simulation GBM
        gbm_sim = gbm(n_years=n_years, n_scenarios=n_scenarios, mu=mu, sigma=sigma,
                      steps_per_year=steps_per_year, s_0=s_0)
        
        # Appliquer la stratégie CPPI
        portfolio_values = cppi(gbm_sim, floor=floor, multiplier=multiplier, 
                                risk_free_rate=rf_rate, rebalance_per_year=rebalance_per_year)
        
        # Calculer la moyenne du portefeuille CPPI
        cppi_mean = portfolio_values.mean(axis=1)
        
        # Tracer les résultats
        plt.figure(figsize=(16, 8))
        
        # Tracer chaque simulation en gris
        plt.plot(portfolio_values, color='grey', linewidth=0.5, alpha=0.25)
        
        # Tracer la ligne du floor
        plt.axhline(y=floor, linestyle='--', color='red', label='Floor')
        
        # Tracer la moyenne du portefeuille CPPI
        plt.plot(cppi_mean, color='blue', alpha=0.75, label='CPPI Portfolio (Mean)')
        
        # Personnaliser le graphique
        plt.title("CPPI Strategy Simulation")
        plt.xlabel("Time Steps")
        plt.ylabel("Portfolio Value")
        plt.grid(True)
        plt.legend()
        plt.show()

# Bind the button to the function
run_button.on_click(run_simulation)

# Display the widgets
display(
    n_years_widget, n_scenarios_widget, mu_widget, sigma_widget,
    steps_per_year_widget, s_0_widget,
    floor_widget, multiplier_widget, rf_rate_widget, rebalance_widget,
    run_button, output
)

IntSlider(value=5, description='Years', max=20, min=1)

IntSlider(value=100, description='Scenarios', max=1000, min=10, step=10)

FloatSlider(value=0.07, description='Mu', max=0.2, step=0.01)

FloatSlider(value=0.15, description='Sigma', max=0.5, step=0.01)

IntSlider(value=255, description='Steps/Year', max=365, min=12)

...

IntSlider(value=4, description='Multiplier', max=10, min=1)

FloatSlider(value=0.04, description='Risk-free Rate', max=0.1, step=0.01)

IntSlider(value=10, description='Rebalance/Year', max=255, min=1)

Button(button_style='success', description='Run Simulation', style=ButtonStyle())

Output()